# Introduction

This notebook continues from the _DataPrep_ notebook and tries to use the FastAI image classification module.

## Data Set

[Qingyi](https://www.kaggle.com/qingyi). (February 2018). WM-811K wafer map, Version 1. Retrieved January 2018 from https://www.kaggle.com/qingyi/wm811k-wafer-map/downloads/wm811k-wafer-map.zip/1.

## License

Copyright 2019 Amazon.com, Inc. or its affiliates. All Rights Reserved.
SPDX-License-Identifier: MIT-0

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

# FastAI

In [ ]:
#!pip install fastai
!pip freeze | grep fastai
!pip freeze | grep scikit

In [ ]:
from fastai.vision import *

## Data loader and transformations

In [ ]:
import torch
import torch.utils.data
import torchvision


class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
    """

    def __init__(self, dataset, indices=None, num_samples=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices
            
        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        dataset_type = type(dataset)
        if dataset_type is torchvision.datasets.MNIST:
            return dataset.train_labels[idx].item()
        elif dataset_type is torchvision.datasets.ImageFolder:
            return dataset.imgs[idx][1]
        else:
            img, label = dataset[idx]
            return label
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples
        


In [ ]:
doc(get_transforms)

In [ ]:
DATA = Path('vdata')
tfms = get_transforms(flip_vert=True, max_lighting = None, max_warp = None, max_rotate = 20.0)
data = ImageDataBunch.from_folder(DATA, ds_tfms=tfms, size=224)

In [ ]:
data.train_dl = data.train_dl.new(shuffle=False, sampler=ImbalancedDatasetSampler(data.train_ds))

In [ ]:
train_ds = data.train_ds
img, label = train_ds[5]
img

In [ ]:
xb,yb = data.one_batch()
xb.shape,yb.shape

In [ ]:
data.show_batch(rows=3, figsize=(5,5))

## Network architecture

In [ ]:
from torch.autograd import Variable

def one_hot_embedding(labels, num_classes):
    return torch.eye(num_classes)[labels.data.cpu()]

# This custom loss function did not help, but leaving it here for reference.
class FocalLoss(nn.Module):

    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps

    def forward(self, input, target):
        y = one_hot_embedding(target, input.size(-1))
        logit = F.softmax(input)
        logit = logit.clamp(self.eps, 1. - self.eps)
        
        loss = -1 * y.cuda() * (torch.log(logit)).cuda() # cross entropy
        loss = loss * (1 - logit) ** self.gamma # focal loss
        return loss.sum(dim=1).mean()



In [ ]:
#learn = Learner(data, model, loss_func = nn.CrossEntropyLoss(), metrics=[accuracy])
#learn = cnn_learner(data, models.resnet18, metrics=accuracy)
#learn = cnn_learner(data, models.resnet34, metrics=accuracy, loss_func = FocalLoss(gamma=2.0))
learn = cnn_learner(data, models.resnet34, metrics=accuracy)

In [ ]:
doc(cnn_learner)

In [ ]:
learn.lr_find(end_lr=1.0)

In [ ]:
learn.recorder.plot()

## Training

* Train the top layers (transfer learning)
* Find a new learning rate
* Unfreeze bottom layers
* Train again (fine tuning)

In [ ]:
doc(fit_one_cycle)

In [ ]:
learn.fit_one_cycle(5, max_lr=0.01)

In [ ]:
learn.save('fastai.res34.imbalance')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-6,5e-5))

It seems that the model is starting to overfit and the accuracy did not improve, so we'll revert to the last snapshot.

In [ ]:
learn.fit_one_cycle(1, max_lr=slice(1e-6,1e-3))

In [ ]:
learn.save('fastai.res34')
learn.export()

## Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
ds = learn.data.train_ds
img, label = ds[5]
learn.predict(img)

In [ ]:
label

In [ ]:
interp.most_confused()

## Test set

In [ ]:
test_data = ImageDataBunch.from_folder(DATA, ds_tfms=tfms, size=224, valid='test')
test_data

In [ ]:
label_map = {}
for cat in test_data.valid_ds.y:
    label_map[cat.data] = cat.obj

In [ ]:
label_map

In [ ]:
from collections import OrderedDict
sorted_labels = OrderedDict(sorted(label_map.items()))
sorted_label_list = list(sorted_labels.values())
sorted_label_list

In [ ]:
from sklearn.metrics import *
test_ds = test_data.valid_ds
preds = []
trues = []
for img, label in test_ds:
    pred = learn.predict(img)
    preds.append(pred[0].data)
    trues.append(label.data)

In [ ]:
print("Accuracy: {0}".format(accuracy_score(trues, preds)))

In [ ]:
print("Weighted F1 Score: {0}".format(f1_score(trues, preds, average='weighted')))

In [ ]:
print("Weighted F-beta: {0}".format(fbeta_score(trues, preds, average='weighted', beta=1.0)))

In [ ]:
print("Macro F1 Score: {0}".format(f1_score(trues, preds, average='macro')))

In [ ]:
print("Macro F-beta: {0}".format(fbeta_score(trues, preds, average='macro', beta=1.0)))

In [ ]:
print("Micro F1 Score: {0}".format(f1_score(trues, preds, average='micro')))

In [ ]:
print("Micro F-beta: {0}".format(fbeta_score(trues, preds, average='micro', beta=1.0)))

In [ ]:
print(classification_report(trues, preds, target_names=sorted_label_list, labels=list(range(len(sorted_label_list)))))

In [ ]:
from sklearn.metrics import precision_score, recall_score
print(precision_score(trues, preds, average='weighted'))
print(recall_score(trues, preds, average='weighted'))

In [ ]:
cm = confusion_matrix(trues, preds)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.figure(figsize=(20,10)) 
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    plt.tight_layout()

In [ ]:
plot_confusion_matrix(cm, sorted_label_list, normalize=False)

In [ ]:
plot_confusion_matrix(cm, sorted_label_list, normalize=True)

Now we should consider the metrics for a binary classification case, considering only whether we identified any pattern at all.

In [ ]:
binary_trues = [ 0 if x == 8 else 1 for x in trues]

In [ ]:
binary_preds = [ 0 if x == 8 else 1 for x in preds]

In [ ]:
print(classification_report(binary_trues, binary_preds, target_names = ['No Defect', 'Defect']))

In [ ]:
print(accuracy_score(binary_trues, binary_preds))

In [ ]:
print(precision_score(binary_trues, binary_preds))
print(recall_score(binary_trues, binary_preds))